# Topic modelling of news headlines for prediction of news category with 7 topics¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

import nlp_preprocessors

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\certifi\core.py:36: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  _CACERT_CTX = get_path("certifi", "cacert.pem")


## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Clean text

In [7]:
train['ti_de_te_cleaned'] = train['title_description_text'].apply(lambda x: nlp_preprocessors.clean_text(x))

## Extract features from 'title_description_text' 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [10]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['ti_de_te_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [11]:
dtm

<67513x29994 sparse matrix of type '<class 'numpy.int64'>'
	with 1026120 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [14]:
len(cv.get_feature_names_out())

29994

In [15]:
cv.get_feature_names_out()

array(['000', '01', '034', ..., 'đoković', 'войны', 'украинцы'],
      dtype=object)

In [16]:
len(lda.components_)

7

In [17]:
lda.components_

array([[14.65800592,  0.14285722,  5.14877427, ...,  0.14285719,
         0.14285767,  0.1428576 ],
       [19.91614703,  0.14285721,  6.77889673, ...,  0.14285718,
         0.1428576 ,  0.14285755],
       [ 2.9412196 ,  0.14285721,  9.52294605, ...,  0.14285718,
         4.14285449,  0.14285753],
       ...,
       [ 3.37662323,  4.14210564, 12.3924028 , ...,  3.14280178,
         0.14285754,  5.14285478],
       [13.63719284,  0.14312224,  0.14305689, ...,  0.14285718,
         0.14285765,  0.14285758],
       [ 6.32692489,  0.14320768, 35.58916322, ...,  0.14285718,
         0.14285757,  0.14285751]])

In [18]:
len(lda.components_[0])

29994

### Show most important words of first extracted topic

In [19]:
first_topic = lda.components_[0]

In [20]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([25046,   922, 23851, ...,  4305, 16450,  5532], dtype=int64)

In [21]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [22]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

polen
zahl
bild
faeser
steigen
geben
groß
britisch
mensch
deutschland


### Show most important words of first extracted topic

In [23]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['zeigen', 'hoch', 'johnson', 'deutsch', 'leben', 'polen', 'zahl', 'bild', 'faeser', 'steigen', 'geben', 'groß', 'britisch', 'mensch', 'deutschland']


Die TOP-15 Wörter für das Thema #1
['million', 'europa', 'gut', 'zeigen', 'land', 'geben', 'taiwan', 'hoch', 'groß', 'deutsch', 'usa', 'euro', 'deutschland', 'us', 'china']


Die TOP-15 Wörter für das Thema #2
['tot', 'tote', 'verletzen', 'geben', 'donald', 'protest', 'mann', 'präsident', 'gericht', 'polizei', 'mensch', 'ex', 'sterben', 'trump', 'us']


Die TOP-15 Wörter für das Thema #3
['liveblog', 'entwicklung', 'biden', 'moskau', 'scholz', 'selenskyj', 'kiew', 'eu', 'ukrainisch', 'präsident', 'putin', 'russisch', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #4
['deutschland', 'geben', 'deutsch', 'gut', 'pandemie', 'gewinnen', 'lauterbach', 'partei', 'bayern', 'wahl', 'berlin', 'afd', 'online', 'focus', 'corona']


Die TOP-15 Wörter für das Thema #5
['un', 'fußball', 'türkei', '

### Get topics with highest probability for news items in train data

In [24]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [25]:
topic_results[0].round(5)

array([0.00624, 0.44526, 0.00622, 0.00623, 0.00623, 0.00622, 0.52359])

In [26]:
topic_results[0].argmax()

6

In [27]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [28]:
train.head()

,date,title,description,category,text,source,title_description_text,ti_de_te_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,6
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,3
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,6
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben rund Mädch...,3
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,5


## Visualize model

In [29]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3      -0.395966  71.688202       1        1  18.306135
4     -23.083799  20.324343       2        1  15.556034
6      32.644588  26.325420       3        1  14.956483
2     -78.898643  14.251719       4        1  13.667792
1      10.016418 -24.915483       5        1  13.435952
5     -56.189262  65.494522       6        1  12.311534
0     -45.664780 -30.938454       7        1  11.766070, topic_info=           Term          Freq         Total Category  logprob  loglift
25499   ukraine  11732.000000  11732.000000  Default  30.0000  30.0000
14352     krieg   5907.000000   5907.000000  Default  29.0000  29.0000
21153  russland   5957.000000   5957.000000  Default  28.0000  28.0000
21147  russisch   5618.000000   5618.000000  Default  27.0000  27.0000
4946      china   2856.000000   2856.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
5527    deutsch    533.250306   4579.807352   Topic7  -5.5511  -0.0105
3344     berlin    423.238865   2780.322471   Topic7  -5.7822   0.2576
13441      kind    367.387570   1407.838314   Topic7  -5.9237   0.7966
11478     heute    365.343249   1815.997621   Topic7  -5.9293   0.5364
14878      land    366.736756   3820.766352   Topic7  -5.9255  -0.2036

[496 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
83         5  0.993150            49
301        1  0.013207  abhängigkeit
301        5  0.983920  abhängigkeit
627        2  0.976838           afd
627        3  0.022527           afd
...      ...       ...           ...
29061      6  0.058279        zeigen
29061      7  0.192975        zeigen
29214      5  0.975075          zins
29214      7  0.024225          zins
29330      4  0.988868    zugunglück

[1057 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 7, 3, 2, 6, 1])

## Save train set with topics and dump countvectorizer and model

In [30]:
#Save result to csv
train.to_csv('train_lda_combined.csv')

In [31]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv_combined.jl')

['models/cv_combined.jl']

In [32]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_7_topics_combined.jl')

['models/lda_model_7_topics_combined.jl']